In [1]:
import sys, os
import pandas as pd
import numpy as np
import csv
import shutil

In [2]:
def append_to_binary(file_name, data):
    #print(file_name)
    with open(file_name, 'ab') as bin_file:
        bin_file.write(data)

def append_line_to_csv(file_name, line):
    with open(file_name, 'a') as csv_file:
        writer = csv.writer(csv_file)
        writer.writerow(line)

def append_batch_to_csv(file_name, data):
    #print(file_name)
    with open(file_name, 'a') as csv_file:
        writer = csv.writer(csv_file)
        writer.writerows(data)


In [3]:
trainers = [5,12,9]
testers = list(range(1, 18+1))
[testers.remove(a) for a in trainers]

#file_list = ['HuGaDB_v1_walking_04_05.txt']
file_list = os.listdir()
file_list = list(filter(lambda x: x.split('.')[-1] == 'txt', file_list))
file_list = sorted(file_list)
ACCEL_FIX = 9.80665*2.0/32768 # m/s^2
GYRO_FIX = 2000/32768 # deg/sec

SENS = ['acc', 'gyro']
LOCS = ['rf','rs','rt','lf','ls','lt']
ACTS = ['bicycling','running','sitting','standing','walking']
DIMS = ['x','y','z']

processed_dir = 'processed'
activity_dir = os.path.join(processed_dir, 'activity')
segment_dir = os.path.join(processed_dir, 'segment')
subject_dir = os.path.join(processed_dir, 'subject')
training_dir = os.path.join(processed_dir, 'training')
file_dir = os.path.join(processed_dir, 'files')

try:
    shutil.rmtree(processed_dir)
except FileNotFoundError:
    pass
os.mkdir(processed_dir)
os.mkdir(activity_dir)
[os.mkdir(os.path.join(activity_dir, a)) for a in ACTS]
os.mkdir(segment_dir)
[os.mkdir(os.path.join(segment_dir, a)) for a in LOCS]
os.mkdir(subject_dir)
[os.mkdir(os.path.join(subject_dir, str(a))) for a in testers]
os.mkdir(training_dir)
os.mkdir(file_dir)
[os.mkdir(os.path.join(file_dir, file_name)) for file_name in file_list]

for data_file in file_list:
    print(data_file)
    data = pd.read_csv(data_file, sep='\t')
    subject = int(data_file.split('_')[-2])
    activity = data_file.split('_')[2]
    for loc in LOCS:
        sensor_data  = []
        human_readable_data = []
        for sen in SENS:
            for dim in DIMS:
                one_d = data['{}_{}_{}'.format(sen, loc, dim)].values
                if sen=='acc':
                    human_readable_data.append(one_d*ACCEL_FIX)
                    sensor_data.append(one_d/4)
                    one_d = one_d / 4 # The acceleraton sensor doesn't use the two LS bits
                if sen=='gyro':
                    sensor_data.append(one_d)
                    human_readable_data.append(one_d*GYRO_FIX)

                data_file_dir = os.path.join(file_dir, data_file)
                append_to_binary(os.path.join(data_file_dir, "{}_{}_{}".format(loc, sen, dim)), one_d.astype(np.int16))

                if subject in trainers:
                    append_to_binary(os.path.join(training_dir, "{}_{}".format(sen, dim)), one_d.astype(np.int16))
                else:
                    # all test
                    append_to_binary(os.path.join(processed_dir, "{}_{}".format(sen, dim)), one_d.astype(np.int16))
                    # activity
                    if activity in ACTS:
                        append_to_binary(os.path.join(activity_dir, activity, "{}_{}".format(sen, dim)), one_d.astype(np.int16))
                    # segment
                    append_to_binary(os.path.join(segment_dir, loc, "{}_{}".format(sen, dim)), one_d.astype(np.int16))
                    # subject
                    append_to_binary(os.path.join(subject_dir, str(subject), "{}_{}".format(sen, dim)), one_d.astype(np.int16))
          
        if subject in trainers:
            append_batch_to_csv(os.path.join(training_dir,'all.csv'), zip(*human_readable_data))
        else:
            append_batch_to_csv(os.path.join(processed_dir, 'all.csv'), zip(*human_readable_data))
            if activity in ACTS:
                append_batch_to_csv(os.path.join(activity_dir, activity, 'all.csv'), zip(*human_readable_data))
            append_batch_to_csv(os.path.join(segment_dir, loc, 'all.csv'), zip(*human_readable_data))
            append_batch_to_csv(os.path.join(subject_dir, str(subject), 'all.csv'), zip(*human_readable_data))

        if subject in trainers:
            append_batch_to_csv(os.path.join(training_dir,'all2.csv'), zip(*sensor_data))
        else:
            append_batch_to_csv(os.path.join(processed_dir, 'all2.csv'), zip(*sensor_data))
            if activity in ACTS:
                append_batch_to_csv(os.path.join(activity_dir, activity, 'all2.csv'), zip(*sensor_data))
            append_batch_to_csv(os.path.join(segment_dir, loc, 'all2.csv'), zip(*sensor_data))
            append_batch_to_csv(os.path.join(subject_dir, str(subject), 'all2.csv'), zip(*sensor_data))

        




HuGaDB_v1_bicycling_01_00.txt
HuGaDB_v1_bicycling_01_01.txt
HuGaDB_v1_bicycling_01_02.txt
HuGaDB_v1_bicycling_01_03.txt
HuGaDB_v1_bicycling_01_04.txt
HuGaDB_v1_bicycling_01_05.txt
HuGaDB_v1_bicycling_01_06.txt
HuGaDB_v1_bicycling_01_07.txt
HuGaDB_v1_bicycling_01_08.txt
HuGaDB_v1_bicycling_01_09.txt
HuGaDB_v1_bicycling_01_10.txt
HuGaDB_v1_bicycling_01_11.txt
HuGaDB_v1_bicycling_01_12.txt
HuGaDB_v1_bicycling_01_13.txt
HuGaDB_v1_bicycling_01_14.txt
HuGaDB_v1_bicycling_01_15.txt
HuGaDB_v1_bicycling_01_16.txt
HuGaDB_v1_bicycling_01_17.txt
HuGaDB_v1_bicycling_01_18.txt
HuGaDB_v1_down_by_elevator_12_00.txt
HuGaDB_v1_running_03_00.txt
HuGaDB_v1_running_03_01.txt
HuGaDB_v1_running_07_00.txt
HuGaDB_v1_running_07_01.txt
HuGaDB_v1_running_07_02.txt
HuGaDB_v1_running_08_00.txt
HuGaDB_v1_running_08_01.txt
HuGaDB_v1_running_08_02.txt
HuGaDB_v1_running_08_03.txt
HuGaDB_v1_running_09_00.txt
HuGaDB_v1_running_09_01.txt
HuGaDB_v1_running_09_02.txt
HuGaDB_v1_sitting_02_00.txt
HuGaDB_v1_sitting_02_01.txt
H

HuGaDB_v1_various_06_11.txt
HuGaDB_v1_various_06_12.txt
HuGaDB_v1_various_06_13.txt
HuGaDB_v1_various_06_14.txt
HuGaDB_v1_various_06_15.txt
HuGaDB_v1_various_06_16.txt
HuGaDB_v1_various_06_17.txt
HuGaDB_v1_various_06_18.txt
HuGaDB_v1_various_06_19.txt
HuGaDB_v1_various_06_20.txt
HuGaDB_v1_various_06_21.txt
HuGaDB_v1_various_06_22.txt
HuGaDB_v1_various_06_23.txt
HuGaDB_v1_various_06_24.txt
HuGaDB_v1_various_06_25.txt
HuGaDB_v1_various_06_26.txt
HuGaDB_v1_various_06_27.txt
HuGaDB_v1_various_07_00.txt
HuGaDB_v1_various_07_01.txt
HuGaDB_v1_various_07_02.txt
HuGaDB_v1_various_07_03.txt
HuGaDB_v1_various_07_04.txt
HuGaDB_v1_various_07_05.txt
HuGaDB_v1_various_07_06.txt
HuGaDB_v1_various_07_07.txt
HuGaDB_v1_various_07_08.txt
HuGaDB_v1_various_07_09.txt
HuGaDB_v1_various_07_10.txt
HuGaDB_v1_various_07_11.txt
HuGaDB_v1_various_07_12.txt
HuGaDB_v1_various_07_13.txt
HuGaDB_v1_various_07_14.txt
HuGaDB_v1_various_07_15.txt
HuGaDB_v1_various_07_16.txt
HuGaDB_v1_various_07_17.txt
HuGaDB_v1_various_07

HuGaDB_v1_walking_08_02.txt
HuGaDB_v1_walking_09_00.txt
HuGaDB_v1_walking_09_01.txt
HuGaDB_v1_walking_09_02.txt
HuGaDB_v1_walking_09_03.txt
HuGaDB_v1_walking_09_04.txt
HuGaDB_v1_walking_10_00.txt
HuGaDB_v1_walking_10_01.txt
HuGaDB_v1_walking_10_02.txt
HuGaDB_v1_walking_10_03.txt
HuGaDB_v1_walking_11_00.txt
HuGaDB_v1_walking_11_01.txt
HuGaDB_v1_walking_11_02.txt
HuGaDB_v1_walking_11_03.txt
HuGaDB_v1_walking_12_00.txt
HuGaDB_v1_walking_12_01.txt
HuGaDB_v1_walking_12_02.txt
HuGaDB_v1_walking_12_03.txt
HuGaDB_v1_walking_12_04.txt
HuGaDB_v1_walking_12_05.txt
HuGaDB_v1_walking_12_06.txt
HuGaDB_v1_walking_13_00.txt
HuGaDB_v1_walking_13_01.txt
HuGaDB_v1_walking_13_02.txt
HuGaDB_v1_walking_13_03.txt
HuGaDB_v1_walking_14_00.txt
HuGaDB_v1_walking_14_01.txt
HuGaDB_v1_walking_14_02.txt
HuGaDB_v1_walking_14_03.txt
HuGaDB_v1_walking_14_04.txt
HuGaDB_v1_walking_15_00.txt
HuGaDB_v1_walking_15_01.txt
HuGaDB_v1_walking_15_02.txt
HuGaDB_v1_walking_15_03.txt
HuGaDB_v1_walking_15_04.txt
HuGaDB_v1_walking_15

In [5]:
trainers = [5,12,9]
testers = list(range(1, 18+1))
[testers.remove(a) for a in trainers]

file_list = ['HuGaDB_v1_walking_06_04.txt']
#file_list = os.listdir()
file_list = list(filter(lambda x: x.split('.')[-1] == 'txt', file_list))
ACCEL_FIX = 9.80665*2.0/32768 # m/s^2
GYRO_FIX = 2000/32768 # deg/sec

SENS = ['acc', 'gyro']
#LOCS = ['rf','rs','rt','lf','ls','lt']
LOCS = ['rf']
ACTS = ['bicycling','running','sitting','standing','walking']
DIMS = ['x','y','z']

processed_dir = 'processed'
activity_dir = os.path.join(processed_dir, 'activity')
segment_dir = os.path.join(processed_dir, 'segment')
subject_dir = os.path.join(processed_dir, 'subject')
training_dir = os.path.join(processed_dir, 'training')

try:
    shutil.rmtree(processed_dir)
except FileNotFoundError:
    pass
os.mkdir(processed_dir)
os.mkdir(activity_dir)
[os.mkdir(os.path.join(activity_dir, a)) for a in ACTS]
os.mkdir(segment_dir)
[os.mkdir(os.path.join(segment_dir, a)) for a in LOCS]
os.mkdir(subject_dir)
[os.mkdir(os.path.join(subject_dir, str(a))) for a in testers]
os.mkdir(training_dir)

data = pd.read_csv(file_list[0],sep='\t')

for data_file in file_list:
    print(data_file)
    subject = int(data_file.split('_')[-2])
    activity = data_file.split('_')[2]
    for loc in LOCS:
        sensor_data  = []
        for sen in SENS:
            for dim in DIMS:
                one_d = data['{}_{}_{}'.format(sen, loc, dim)].values
                if sen=='acc':
                    sensor_data.append(one_d*ACCEL_FIX)
                if sen=='gyro':
                    sensor_data.append(one_d*GYRO_FIX)
                    
                if subject in trainers:
                    append_to_binary(os.path.join(training_dir, "{}_{}".format(sen, dim)), one_d.astype(np.int16))
                else:
                    # all test
                    append_to_binary(os.path.join(processed_dir, "{}_{}".format(sen, dim)), one_d.astype(np.int16))
                    # activity
                    if activity in ACTS:
                        append_to_binary(os.path.join(activity_dir, activity, "{}_{}".format(sen, dim)), one_d.astype(np.int16))
                    # segment
                    append_to_binary(os.path.join(segment_dir, loc, "{}_{}".format(sen, dim)), one_d.astype(np.int16))
                    # subject
                    append_to_binary(os.path.join(subject_dir, str(subject), "{}_{}".format(sen, dim)), one_d.astype(np.int16))
          
        if subject in trainers:
            append_batch_to_csv(os.path.join(training_dir,'all.csv'), zip(*sensor_data))
        else:
            append_batch_to_csv(os.path.join(processed_dir, 'all.csv'), zip(*sensor_data))
            if activity in ACTS:
                append_batch_to_csv(os.path.join(activity_dir, activity, 'all.csv'), zip(*sensor_data))
            append_batch_to_csv(os.path.join(segment_dir, loc, 'all.csv'), zip(*sensor_data))
            append_batch_to_csv(os.path.join(subject_dir, str(subject), 'all.csv'), zip(*sensor_data))

        




HuGaDB_v1_walking_06_04.txt


In [10]:
file_list[0]

'HuGaDB_v1_standing_07_03.txt'

In [11]:
data = pd.read_csv(file_list[0], sep='\t')

In [13]:
data['gyro_rf_z'].values

array([ 13,   7,   7, ..., -18, -23, -21])